# Species classifier based on codon signatire

### Libraries

In [78]:
import pandas as p
from fastai import *

from fastai.imports import*
from fastai.structured import *
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
from sklearn import metrics

from treeinterpreter import treeinterpreter as ti

### Dataset

In [2]:
PATH = "/home/zanarfa/proj/codon_signature/data/vibrio_data/"

In [3]:
! ls -l {PATH}

total 668
drwxr-xr-x 2 zanarfa zanarfa   4096 Nov 15 15:52 final
-rw-r--r-- 1 zanarfa zanarfa    388 Nov 12 11:10 readme.txt
drwxr-xr-x 2 zanarfa zanarfa   4096 Nov 15 15:52 step1
drwxr-xr-x 2 zanarfa zanarfa   4096 Nov 15 15:52 step2
-rw-r--r-- 1 zanarfa zanarfa 648535 Nov 12 11:09 vibrio_original_refseq.csv
-rw-r--r-- 1 zanarfa zanarfa    216 Nov 15 16:16 vibrio_taxid_9.csv
-rw-r--r-- 1 zanarfa zanarfa  10391 Okt 28 17:09 vibrio_taxid.xlsx


#### taxonID/species (as dict and id list)

In [4]:
taxon_species={}

with open (f'{PATH}vibrio_taxid_9.csv', "r") as tax_sp:
    for line in tax_sp:
        line = line.split(",")
        taxonID = line[0]
        species = line[1].replace(" ","_").strip("\n")
        taxon_species[taxonID]=species

In [5]:
taxon_species

{'663': 'Vibrio_alginolyticus',
 '666': 'Vibrio_cholerae',
 '669': 'Vibrio_harveyi',
 '670': 'Vibrio_parahaemolyticus',
 '672': 'Vibrio_vulnificus',
 '676': 'Vibrio_fluvialis',
 '680': 'Vibrio_campbellii',
 '685': 'Vibrio_diazotrophicus',
 '689': 'Vibrio_mediterranei'}

In [6]:
ID_list=[x for x in taxon_species.keys()]

#### data from step 2 as test data (as dictionary of df)

In [161]:
vibrio_originals = {}

for id in ID_list: 
    vibrio_originals[str(taxon_species[id])+"_original"] = pd.read_csv(f'{PATH}step2/{id}_step2_res.tsv', 
                                                                      low_memory=False, sep="\t")

Example of "Vibrio_mediterranei" table in **vibrio_originals** dictionary

In [162]:
vibrio_originals["Vibrio_mediterranei_original"]

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32
0,0.56,0.5,0.58,0.65,0.53,0.77,0.13,0.23,0.39,0.32,...,0.64,0.59,0.88,0.49,0.44,0.97,0.55,0.93,0.99,0.67
1,0.56,0.5,0.57,0.66,0.52,0.76,0.13,0.23,0.39,0.32,...,0.64,0.60,0.87,0.49,0.44,0.97,0.54,0.93,0.99,0.67
2,0.56,0.5,0.58,0.66,0.53,0.76,0.13,0.23,0.39,0.32,...,0.64,0.60,0.87,0.49,0.44,0.97,0.54,0.93,0.99,0.67
3,0.56,0.5,0.57,0.66,0.53,0.77,0.13,0.23,0.39,0.32,...,0.64,0.59,0.85,0.49,0.43,0.97,0.54,0.93,0.99,0.67
4,0.57,0.5,0.57,0.66,0.53,0.78,0.13,0.23,0.40,0.33,...,0.64,0.59,0.86,0.49,0.43,0.97,0.54,0.93,0.98,0.67
5,0.56,0.5,0.57,0.66,0.53,0.77,0.13,0.23,0.39,0.32,...,0.64,0.59,0.85,0.49,0.44,0.97,0.54,0.93,0.99,0.66


Add a new column with the species name to each table and merge all species in one df called **vibrio_originals_df**.

In [172]:
vibrio_originals_df = pd.DataFrame()

for species in vibrio_originals.keys():
    species_name=species.replace("_original", "")
    vibrio_originals[species]["species"]=species_name
    
    vibrio_originals_df = pd.concat([vibrio_originals_df, vibrio_originals[species]], ignore_index=True)

#### 9 species, 10k lines each (as dictionary of df)

Load trainig and validation data, store them in a dict, split train and validation, create a single df

In [10]:
#dictionary for all 9 species (10k lines each)
vibrio_10k = {}

for id in ID_list: 
    vibrio_10k[str(taxon_species[id])+"_10k"] = pd.read_csv(f'{PATH}final/{id}_final.tsv', 
                                                                      low_memory=False, sep="\t")
    
#Dataframe for training ad validation
vibrio_10k_train_df = pd.DataFrame()
vibrio_10k_val_df = pd.DataFrame()
#9000 lines for training and 1000 for validating
n = 9000


for species in vibrio_10k.keys():
    #add new column for species name
    species_name=species.replace("_10k", "")
    vibrio_10k[species]["species"]=species_name
    
    #split train and val within the species 
    train = vibrio_10k[species][:n]
    val = vibrio_10k[species][n:]
    
    #train and cal general dataset
    vibrio_10k_train_df = pd.concat([vibrio_10k_train_df, train])
    vibrio_10k_val_df = pd.concat([vibrio_10k_val_df, val])

Split independent and dependent vatiable

In [11]:
x_train, y_train, nas = proc_df(vibrio_10k_train_df, 'species')

In [12]:
x_valid, y_valid, nas = proc_df(vibrio_10k_val_df, 'species')

Create a dictionary to keep trak of the species to cardinality correspondance

In [13]:
species_to_cardinality = {}
tmp_list=[]

for i in taxon_species.values():
    tmp_list.append(i)
    
tmp_list = sorted(tmp_list)

idx=0
for sp in tmp_list:
    species_to_cardinality[sp]=idx
    idx = idx + 1

#### Compute the model

In [14]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(x_train), y_train), rmse(m.predict(x_valid), y_valid),
                m.score(x_train, y_train), m.score(x_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [71]:
m = RandomForestClassifier(n_estimators=10, max_depth=20, bootstrap=True, n_jobs=-1)

Split independent and dependent vatiable

In [72]:
x_train, y_train, nas = proc_df(vibrio_10k_train_df, 'species')

In [73]:
x_valid, y_valid, nas = proc_df(vibrio_10k_val_df, 'species')

Fit the model

In [74]:
m.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=20, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
                       oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [75]:
print_score(m)

[0.0, 0.0, 1.0, 1.0]


**It seems is working too good. Let's try with a tiny trainig**

In [20]:
m1 = RandomForestClassifier(n_estimators=1, max_depth=5, bootstrap=False, n_jobs=-1)

In [21]:
x_train, y_train, nas = proc_df(vibrio_10k_train_df.head(), 'species')

In [298]:
x_train

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32
0,0.56,0.39,0.54,0.59,0.60,0.85,0.1,0.23,0.42,0.30,...,0.64,0.46,0.73,0.45,0.49,0.87,0.54,0.87,0.98,0.66
1,0.56,0.39,0.53,0.59,0.60,0.85,0.1,0.23,0.42,0.29,...,0.64,0.45,0.72,0.44,0.49,0.87,0.54,0.88,0.97,0.66
2,0.56,0.38,0.53,0.59,0.58,0.84,0.1,0.23,0.41,0.30,...,0.64,0.45,0.73,0.45,0.49,0.86,0.54,0.88,0.97,0.65
3,0.56,0.40,0.53,0.60,0.59,0.86,0.1,0.22,0.42,0.30,...,0.64,0.45,0.73,0.45,0.48,0.87,0.54,0.88,0.97,0.66
4,0.56,0.39,0.54,0.58,0.61,0.85,0.1,0.23,0.42,0.31,...,0.64,0.45,0.73,0.45,0.50,0.85,0.54,0.88,0.97,0.66


In [22]:
m1.fit(x_train, y_train)

RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
                       max_depth=5, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1, n_jobs=-1,
                       oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [23]:
print_score(m1)

[0.0, 4.760952285695233, 1.0, 0.1111111111111111]


The reason is not working now is because using the firsts 10 lines means train only on 1 species.  
Let's try with more rows and randomly sampled.

In [107]:
m2 = RandomForestClassifier(n_estimators=1, max_depth=5, bootstrap=False, n_jobs=-1)

In [108]:
x_train, y_train, nas = proc_df(vibrio_10k_train_df.sample(100), 'species')

In [109]:
m2.fit(x_train, y_train)

RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
                       max_depth=5, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1, n_jobs=-1,
                       oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [110]:
print_score(m2)

[2.8460498941515415, 3.186778240724565, 0.74, 0.6666666666666666]


Now with bootstrapping and more trees to get also OOB score

In [28]:
m3 = RandomForestClassifier(n_estimators=10, max_depth=5, bootstrap=True, n_jobs=-1, oob_score=True)

In [29]:
x_train, y_train, nas = proc_df(vibrio_10k_train_df.sample(1000), 'species')

In [30]:
m3.fit(x_train, y_train)

/home/zanarfa/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/home/zanarfa/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: divide by zero encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/home/zanarfa/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=5, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
                       oob_score=True, random_state=None, verbose=0,
                       warm_start=False)

In [31]:
print_score(m3)

[0.0, 0.0, 1.0, 1.0, 0.961]


#### Tree interpreter to get proportion for each species, depending on the model

In [86]:
species_to_cardinality

{'Vibrio_alginolyticus': 0,
 'Vibrio_campbellii': 1,
 'Vibrio_cholerae': 2,
 'Vibrio_diazotrophicus': 3,
 'Vibrio_fluvialis': 4,
 'Vibrio_harveyi': 5,
 'Vibrio_mediterranei': 6,
 'Vibrio_parahaemolyticus': 7,
 'Vibrio_vulnificus': 8}

In [137]:
row = x_valid.values[None,0]
y_valid[0]

0

In [103]:
prediction, bias, contributions = ti.predict(m, row)
prediction[0]

array([1., 0., 0., 0., 0., 0., 0., 0., 0.])

In [104]:
prediction, bias, contributions = ti.predict(m1, row)
prediction[0]

array([1.])

In [115]:
prediction, bias, contributions = ti.predict(m2, row)
prediction[0]

array([0.25   , 0.22222, 0.     , 0.     , 0.25   , 0.     , 0.     , 0.27778, 0.     ])

In [116]:
prediction, bias, contributions = ti.predict(m3, row)
prediction[0]

array([0.73977, 0.07324, 0.     , 0.     , 0.     , 0.03355, 0.07199, 0.05468, 0.02676])

### Test on real data

In [117]:
vibrio_originals_df.shape

(1398, 33)

In [118]:
x_valid, y_valid, nas = proc_df(vibrio_originals_df, 'species')

In [120]:
print_score(m)

[0.0, 0.3782347372361169, 1.0, 0.9928469241773963]


Now I'm sure that model m is working because the results are not perfect using real data (orifinal ratios from step 2).

#### Distribution of the prediction

In [126]:
predictions = {}

for i in species_to_cardinality.keys():
    predictions[i]=[]

In [177]:
for i in range(len(vibrio_originals_df)):
    row = vibrio_originals_df.drop('species', axis=1).values[None,i]
    prediction, bias, contributions = ti.predict(m, row)
    species = vibrio_originals_df.loc[i, "species"]
    idx = species_to_cardinality[species]
    predictions[species].append(prediction[0][idx])
    
    

In [179]:
predictions

{'Vibrio_alginolyticus': [1.0,
  1.0,
  1.0,
  1.0,
  0.1,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.9,
  0.7,
  1.0,
  1.0,
  1.0,
  1.0,
  0.7,
  1.0,
  1.0,
  0.3,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.8,
  0.8,
  0.9],
 'Vibrio_campbellii': [0.8,
  0.8,
  1.0,
  0.8,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.8,
  0.4,
  1.0,
  1.0,
  0.8,
  0.4,
  0.5,
  1.0,
  0.8,
  1.0,
  1.0,
  0.7,
  0.8],
 'Vibrio_cholerae': [0.9,
  0.9,
  1.0,
  0.9,
  1.0,
  0.9,
  1.0,
  0.9,
  1.0,
  1.0,
  0.9,
  1.0,
  1.0,
  0.9,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.9,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.9,
  1.0,
  1.0,
  1.0,
  0.9,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.9,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.9,
  1.0,
  1.0,
  0.9,
  1.0,
  1.0,
  0.9,
  1.0,
  1.0,
  1